In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

# Figure 3

In [ ]:
all_elec_models = {}

for COP in [3, 5.5, 8]:
    y_all = f"all_elec_COP_{COP}"
    all_elec_models[COP] = {}

    f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
    ax = ax.flatten()

    for i, project in enumerate(pu.PROJECTS):
        data[project][y_all] = (
            data[project]["cooling"] * pu.cop_to_kWe_per_MJc(COP)
            + data[project]["electricity"]
        )
        weekend = pu.choose_weekend[project]
        pu.plot_base(
            data[project],
            fax=(f, ax[i]),
            weekend=weekend,
            y=y_all,
            ylabel=f"Total electric load (COP: {COP})\n(kWh/m$^2$/day)",
        )
        all_elec_models[COP][project] = pu.create_model(
            data[project].dropna(), fax=(f, ax[i]), weekend=weekend, y=y_all, log=True
        )
        ax[i].set_title(f"{pu.plot_numbers[i]}) {project}")

    for a in ax:
        a.xaxis.set_tick_params(labelbottom=True)
    f.tight_layout()
    pu.add_legend((f, ax))

    if COP != 5.5:
        plt.close(f)
    else:
        if pu.save_fig:
            f.savefig(pu.fig_path / f"Figure 3 A-F.pdf")

In [ ]:
ms = [1, 1, 1, 1, 1]
f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.8))
ax = ax.flatten()

j = -0.1
for iq, q in enumerate(all_elec_models.keys()):
    verb = q == 5.5
    for i, project in enumerate(pu.PROJECTS):
        if verb:
            print(project)
        pu.ci_plot(
            ax[0],
            i + j,
            all_elec_models[q][project],
            "Treated",
            color=pu.COLORS[iq],
            ms=ms[iq],
            verb=verb,
        )
        pu.ci_plot(
            ax[1],
            i + j,
            all_elec_models[q][project],
            "Tmean",
            color=pu.COLORS[iq],
            ms=ms[iq],
            verb=verb,
        )
        if verb:
            print()
    j += 0.1


for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=30, ha="right")
ax[0].set_ylabel("Impact (%)")
ax[1].set_ylabel("Impact (%/F)")

title0 = (
    f"{pu.plot_numbers[6]}) "
    + "Impact of 2$\degree$F set point increase on electric load\n(includes electricity for cooling water)"
)
title1 = (
    f"{pu.plot_numbers[7]}) "
    + "Impact of mean daily OAT on electric load\n(includes electricity for cooling water)"
)


ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.8, bottom=0.4)

for i, cop in enumerate(all_elec_models.keys()):
    # cop = 1/(val * constants.kW_to_ton)
    ax[-1].plot([], [], "o", color=pu.COLORS[i], label=f"COP: {cop:.1f}", ms=2)
ax[-1].legend(loc=8, bbox_transform=f.transFigure, bbox_to_anchor=(0.5, 0.01), ncol=3)

if pu.save_fig:
    f.savefig(pu.fig_path / "Figure 3 G-H.pdf")

# Figure S2 Remake
Sanity check for notebook on Figure 2

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

elec_cooling_models = {}

for COP in [3, 5.5, 8]:
    y = f"cooling_elec_COP_{COP}"

    elec_cooling_models[COP] = {}

    f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
    ax = ax.flatten()

    for i, project in enumerate(pu.PROJECTS):
        data[project][y] = data[project]["cooling"] * pu.cop_to_kWe_per_MJc(COP)

        weekend = pu.choose_weekend[project]
        pu.plot_base(
            data[project],
            fax=(f, ax[i]),
            weekend=weekend,
            y=y,
            ylabel="Electric load for cooling\n(kWh/m$^2$/day)",
        )
        elec_cooling_models[COP][project] = pu.create_model(
            data[project].dropna(), fax=(f, ax[i]), weekend=weekend, y=y, log=True
        )
        ax[i].set_title(project)

    for a in ax:
        a.xaxis.set_tick_params(labelbottom=True)
    f.tight_layout()
    pu.add_legend((f, ax))

    if COP != 5.5:
        plt.close(f)

# Table S4

In [ ]:
cop = 5.5
with open(pu.si_path / f"Table S4.tex", "w") as fw:
    fw.write(
        pu.si_table(
            all_elec_models[cop],
            pu.PROJECTS,
            caption="Additional numerical data for models shown in Figure 3 (electricity results with cooling).",
            label="table:s4",
        )
    )